In [1]:
import os
os.chdir("..")
os.getcwd()

'/Users/laurasisson/odor-pair'

In [3]:
import analysis.best as best
model = best.get_model()
model

MixturePredictor(
  (gcn): GCN(
    (pad): ZeroPad2d((0, 91, 0, 0))
    (gcn): GINConv(nn=Sequential(
      (0): Sequential(
        (0): Linear(in_features=100, out_features=100, bias=True)
        (1): ReLU()
        (2): Dropout(p=0, inplace=False)
      )
      (1): Sequential(
        (0): Linear(in_features=100, out_features=100, bias=True)
        (1): ReLU()
        (2): Dropout(p=0, inplace=False)
      )
    ))
    (readout): Set2Set(100, 200)
    (post_mp): Sequential(
      (0): Sequential(
        (0): Linear(in_features=200, out_features=100, bias=True)
        (1): ReLU()
        (2): Dropout(p=0, inplace=False)
      )
      (1): Sequential(
        (0): Linear(in_features=100, out_features=100, bias=True)
      )
    )
  )
  (out): Sequential(
    (0): Sequential(
      (0): Linear(in_features=200, out_features=77, bias=True)
      (1): ReLU()
      (2): Dropout(p=0, inplace=False)
    )
    (1): Sequential(
      (0): Linear(in_features=77, out_features=77, bias=True)

In [22]:
import json
import os

folddir = "dataset/folds"

fold_fnames = os.listdir(folddir)
folds = []
for fname in fold_fnames:
    if not fname.endswith(".json"):
        continue
    with open(os.path.join("dataset/folds",fname)) as f:
        folds.append(json.load(f))
folds[0].keys()

dict_keys(['train', 'test', 'validate', 'covered_notes'])

In [24]:
example = folds[0]["train"][0]
example

{'edge': ['CC(=CCCC(C)(C=C)O)C', 'C1CCCCCOCCCCOC(=O)CCCC1'],
 'blend_notes': ['citrus', 'floral']}

In [31]:
import torch
import pairing.data

example_pd = pairing.data.to_pairdata((example["edge"],{n:n for n in example["blend_notes"]}),folds[0]["covered_notes"])
example_pd

PairData(y=[49], x_s=[11, 9], edge_attr_s=[20, 3], edge_index_s=[2, 20], smiles_s='CC(=CCCC(C)(C=C)O)C', x_t=[18, 9], edge_attr_t=[36, 3], edge_index_t=[2, 36], smiles_t='C1CCCCCOCCCCOC(=O)CCCC1')

In [35]:
loader = pairing.data.loader([example_pd],batch_size=1)

In [39]:
for batch in loader:
    print(batch)
    print(model(**batch))

PairDataBatch(y=[1, 49], x_s=[11, 9], x_s_batch=[11], x_s_ptr=[2], edge_attr_s=[20, 3], edge_index_s=[2, 20], smiles_s=[1], x_t=[18, 9], x_t_batch=[18], x_t_ptr=[2], edge_attr_t=[36, 3], edge_index_t=[2, 36], smiles_t=[1])
tensor([[ -7.5304, -10.9434,  -4.1523, -10.5723,  -6.7158,  -5.7163,  -7.3422,
          -5.6530,  -2.2875,  -5.3072,  -6.5789, -12.0449, -11.1216, -10.1371,
         -10.8562,  -7.3053,  -8.9638, -13.4858,  -9.5016,  -6.1817,  -2.5538,
          -7.6263, -14.2503,  -7.3566, -12.8955,  -5.9212, -13.8315,  -5.8680,
          -5.3056,  -5.9917, -14.1736,  -8.8033,  -5.7729,  -9.3598,   0.9498,
          -8.6636,  -2.6678,  -7.0573, -14.2349,  -1.1380,  -2.6668,  -4.8227,
         -10.4803,  -7.6062,  -7.5505, -13.7821,  -3.8134,  -5.7388,  -5.5302,
          -5.6775, -11.0578,  -5.6931, -15.7447, -10.6977,  -6.9568,  -4.9923,
         -14.0037,  -9.9651,  -9.5920, -14.6009,  -5.3259, -16.1524, -10.6952,
          -7.8088, -18.2082, -19.2514,  -3.4855,  -9.0148,  -6.970

# Steps now
### Make train/test/validate dataset for each fold
### Try just doing the AUROC there for now
### Probavbly will have to do the full training :(